
# Projet Maison 2

In [23]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [97]:
# chargement et traitement des données
GEO = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
# A COMPLETER avec les colonnes
# - lat, lon : latitude et longitude des communes
GEO[["lat","lon"]] = GEO["geo_point_2d"].str.split(", ", expand=True)
lat = GEO["lat"]
long = GEO["lon"]
# - cp_ville : Code Postal + " " + "Commune"
cp_ville = GEO["Code Postal"] + " " + GEO["Commune"]
GEO["cp_ville"] = cp_ville
GEO

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,lat,lon,cp_ville
Code INSEE,,,,,,,,,,,,,
31080,31350,BOULOGNE-SUR-GESSE,HAUTE-GARONNE,MIDI-PYRENEES,Chef-lieu canton,301.0,2470.0,1.6,"43.2904403081, 0.650641474176","{""type"": ""Polygon"", ""coordinates"": [[[0.671852...",43.2904403081,0.650641474176,31350 BOULOGNE-SUR-GESSE
11143,11510,FEUILLA,AUDE,LANGUEDOC-ROUSSILLON,Commune simple,314.0,2426.0,0.1,"42.9291375888, 2.90138923544","{""type"": ""Polygon"", ""coordinates"": [[[2.906996...",42.9291375888,2.90138923544,11510 FEUILLA
43028,43200,BESSAMOREL,HAUTE-LOIRE,AUVERGNE,Commune simple,888.0,743.0,0.4,"45.1306448726, 4.07952494849","{""type"": ""Polygon"", ""coordinates"": [[[4.091674...",45.1306448726,4.07952494849,43200 BESSAMOREL
78506,78660,PRUNAY-EN-YVELINES,YVELINES,ILE-DE-FRANCE,Commune simple,155.0,2717.0,0.8,"48.5267627187, 1.80513972814","{""type"": ""Polygon"", ""coordinates"": [[[1.829610...",48.5267627187,1.80513972814,78660 PRUNAY-EN-YVELINES
84081,84310,MORIERES-LES-AVIGNON,VAUCLUSE,PROVENCE-ALPES-COTE D'AZUR,Commune simple,49.0,1042.0,7.6,"43.9337788848, 4.90875878315","{""type"": ""Polygon"", ""coordinates"": [[[4.927176...",43.9337788848,4.90875878315,84310 MORIERES-LES-AVIGNON
80462,80150,LAMOTTE-BULEUX,SOMME,PICARDIE,Commune simple,59.0,613.0,0.4,"50.1865666179, 1.83564836965","{""type"": ""Polygon"", ""coordinates"": [[[1.849459...",50.1865666179,1.83564836965,80150 LAMOTTE-BULEUX
40200,40410,MOUSTEY,LANDES,AQUITAINE,Commune simple,49.0,6736.0,0.7,"44.3882918763, -0.750880080586","{""type"": ""Polygon"", ""coordinates"": [[[-0.71016...",44.3882918763,-0.750880080586,40410 MOUSTEY
17394,17700,SAINT-SATURNIN-DU-BOIS,CHARENTE-MARITIME,POITOU-CHARENTES,Commune simple,45.0,2528.0,0.9,"46.1401428747, -0.665133364912","{""type"": ""Polygon"", ""coordinates"": [[[-0.66124...",46.1401428747,-0.665133364912,17700 SAINT-SATURNIN-DU-BOIS
...,...,...,...,...,...,...,...,...,...,...,...,...,...


**Partie A**

- Compléter le chargement des données en ajoutant au dataframe `GEO`
    - les colonnes "lat" et "lon" avec la latitude et la longitude des communes
    - une colonne "cp_ville" avec le Code Postal + un espace + et le nom de la Commune
- Ecrire une fonction `search_city(lat, lon)` qui retourne le "cp_ville" de la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ecrire une fonction `dms2dec(deg, min, sec)` qui convertit les degrés, minutes, secondes en valeur numérique pour pouvoir utiliser la fonction précédente avec un GPS.

In [204]:
# fonction recherche de ville
def search_city(lat, long):
    GEO["norm"] = GEO.apply(lambda x: np.sqrt(np.square(lat-float(x.lat))+np.square(long-float(x.lon))), axis=1) 
    value = GEO["norm"].idxmin()
    return GEO["cp_ville"][value]

In [205]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + (mn/60) + (sec/3600)

In [206]:
%time
# on applique la fonction à une coordonnée tirée au hasard
np.random.seed(0)
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = np.random.uniform(a, b)
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
lon = np.random.uniform(a, b)
search_city(lat, lon)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 2.15 µs


'71330 BOSJEAN'

In [207]:
# à partir de coordonnées GPS précises
search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'91120 PALAISEAU'

**Partie B**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python **json** pour parser les valeurs de la colonne "geo_shape" et mettre le résultat (`Series`) dans la variable `GEO_SHAPE`.
- Ecrire une fonction `get_types()` qui retourne le décompte (`value_counts()`) des valeurs accédées avec la clé "type".
- Ecrire une fonction `get_coordinates_len()` qui retourne le décompte (`value_counts()`) des longueurs des listes accédées avec la clé "coordinates".
- Ecrire une fonction `get_most_complex_city()` qui retourne la commune est constituée du plus grand nombre de polygones ?
- Ecrire une fonction `get_nb_cities_2_polygons()` qui retourne  le nombre de villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- **Facultatif :**
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : on pourra installer la librairie **shapely**, utiliser la classe Polygon de **shapely.geometry**  et la méthode `contains()`. Sur Windows **shapely** peut nécessiter d'installer manuellement la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [208]:
# GEO_SHAPE

# La variable GEO_SHAPE doit contenir une Serie
# correspondant aux valeurs de la colonne "geo_shape" parsées avec la librairie json
GEO_SHAPE = pd.Series(dtype=object)

In [209]:
# value_counts des valeurs "type"
def get_types():
    pass

In [210]:
# value_counts des longueurs de "coordinates"
def get_coordinates_len():
     pass

In [211]:
# commune constituée du plus grand nombre de polygones
def get_most_complex_city():
    pass

In [212]:
# commune constituée du plus grand nombre de polygones
def get_nb_cities_2_polygons():
    pass

In [213]:
# tests
import unittest

class Session2Test(unittest.TestCase):
    
    def test_partie_A1(self):
        # on applique la fonction cherche_ville() à une coordonnée tirée au hasard
        np.random.seed(0)
        a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
        lat = np.random.uniform(a, b)
        a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
        lon = np.random.uniform(a, b)

        cp_ville = search_city(lat, lon)
        self.assertEqual(cp_ville, "71330 BOSJEAN")
        
    def test_partie_A2(self):
        # à partir de coordonnées GPS précises
        cp_ville = search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))
        self.assertEqual(cp_ville, "91120 PALAISEAU")
        
    def test_partie_B1(self):
        # check types
        dico = get_types()
        self.assertEqual(dico["Polygon"], 36670)
        self.assertEqual(dico["MultiPolygon"], 72)
        
    def test_partie_B2(self):
        # check coordinates len
        dico = get_coordinates_len()
        self.assertEqual(dico[1], 36660)
        self.assertEqual(dico[2], 80)
       
    def test_partie_B3(self):
        # check most complex city
        cp_ville = get_most_complex_city()
        self.assertEqual(cp_ville, "83400 HYERES")
        
    def test_partie_B4(self):
        # check nb cities 2 polygons
        nb = get_nb_cities_2_polygons()
        self.assertEqual(nb, 10)

In [214]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session2Test)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)
    
run_tests()

test_partie_A1 (__main__.Session2Test) ... ok
test_partie_A2 (__main__.Session2Test) ... ok
test_partie_B1 (__main__.Session2Test) ... ERROR
test_partie_B2 (__main__.Session2Test) ... ERROR
test_partie_B3 (__main__.Session2Test) ... FAIL
test_partie_B4 (__main__.Session2Test) ... FAIL

ERROR: test_partie_B1 (__main__.Session2Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_647/3431394153.py", line 25, in test_partie_B1
    self.assertEqual(dico["Polygon"], 36670)
TypeError: 'NoneType' object is not subscriptable

ERROR: test_partie_B2 (__main__.Session2Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_647/3431394153.py", line 31, in test_partie_B2
    self.assertEqual(dico[1], 36660)
TypeError: 'NoneType' object is not subscriptable

FAIL: test_partie_B3 (__main__.Session2Test)
----------------------------------------